# Explore and Cluster Neighborhoods in Toronto
### Scrape wiki page for a table of postal codes and pre process into a dataframe.
### Use dataframe to explore and cluster neighborhoods 

#### Explore Beautifulsoup library for web scraping. 

##### Documentation: https://beautiful-soup-4.readthedocs.io/en/latest/
##### Youtube video: https://www.youtube.com/watch?v=ng2o98k983k

In [1]:
from bs4 import BeautifulSoup
import requests

import pandas as pd

In [2]:
weblink="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

toronto_data=requests.get(weblink)
#toronto_data.text

Now we scrape data for required info (tables)

Begin with find_all 'table' and Get to know class of required table

Use BeautifulSoup to map the table to a variable

In [3]:
soup = BeautifulSoup(toronto_data.text,'lxml')
#print(soup.prettify())
#print(soup.get_text())

#table=soup.find_all('table')
#soup.table['class']
Toronto = soup.find('table',{'class':'wikitable sortable'})
Toronto

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

It can be seen that:
1) Every row is a "tr"
2) Heading in row "th"
3)row data in "td"

In [4]:
toto_dict=[]

for tr in Toronto.find_all('tr')[1:-1]:
    data=tr.find_all(['th','td'])
    pcode=data[0].string
    bor=data[1].string
    neigh=(data[2].text).split("\n")[0]
    #print(neigh)
    
    toto_dict.append({'Postcode':pcode, 'Borough':bor, 'Neighbourhood':neigh})

toto_dict

[{'Postcode': 'M1A',
  'Borough': 'Not assigned',
  'Neighbourhood': 'Not assigned'},
 {'Postcode': 'M2A',
  'Borough': 'Not assigned',
  'Neighbourhood': 'Not assigned'},
 {'Postcode': 'M3A', 'Borough': 'North York', 'Neighbourhood': 'Parkwoods'},
 {'Postcode': 'M4A',
  'Borough': 'North York',
  'Neighbourhood': 'Victoria Village'},
 {'Postcode': 'M5A',
  'Borough': 'Downtown Toronto',
  'Neighbourhood': 'Harbourfront'},
 {'Postcode': 'M5A',
  'Borough': 'Downtown Toronto',
  'Neighbourhood': 'Regent Park'},
 {'Postcode': 'M6A',
  'Borough': 'North York',
  'Neighbourhood': 'Lawrence Heights'},
 {'Postcode': 'M6A',
  'Borough': 'North York',
  'Neighbourhood': 'Lawrence Manor'},
 {'Postcode': 'M7A',
  'Borough': "Queen's Park",
  'Neighbourhood': 'Not assigned'},
 {'Postcode': 'M8A',
  'Borough': 'Not assigned',
  'Neighbourhood': 'Not assigned'},
 {'Postcode': 'M9A',
  'Borough': 'Etobicoke',
  'Neighbourhood': 'Islington Avenue'},
 {'Postcode': 'M1B', 'Borough': 'Scarborough', 'Nei

In [5]:
#Convert to pandas DataFrame
Table=pd.DataFrame.from_dict(toto_dict, orient='columns')
Table=Table[['Postcode','Borough','Neighbourhood']]
print(Table.shape)
Table.head()

(288, 3)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Filter Table for columns having an assigned Borough

In [6]:
Table=Table[Table.Borough != 'Not assigned']
print(Table.shape)
Table.reset_index(drop=True,inplace=True)
Table.head()

(212, 3)


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Now process for:
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 

In [7]:
neigh_none=Table['Neighbourhood'].values.tolist().index('Not assigned')
Table['Neighbourhood'][neigh_none]=Table['Borough'][neigh_none]
Table.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


Merge neighborhoods in the same Borough

In [8]:
Table1=Table.groupby(['Postcode','Borough'])['Neighbourhood'].apply(list).reset_index()
print(Table1.shape)
for ind in range(0,Table1.shape[0]):
    Table1['Neighbourhood'][ind]=",".join(Table1['Neighbourhood'][ind])
Table1

(103, 3)


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


Print shape and top 5 lines of resultant table

In [9]:
print("Table dimensions are:",Table1.shape)

Table1.head()

Table dimensions are: (103, 3)


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Part2: Location data (Geocoder or csv file)

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data



In [10]:
!wget -q -O 'toronto_data.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [11]:
df_geo=pd.read_csv("toronto_data.csv")
df_geo.head()
df_geo.set_axis(['Postcode', 'Latitude', 'Longitude'], axis=1, inplace=True)
df_geo.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
Table_final=Table1.merge(df_geo,on='Postcode',how='left')
print(Table_final.shape)
Table_final

(103, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [13]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(Table_final['Borough'].unique()),
        Table_final.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


### Part 3: Explore and cluster the neighborhoods in Toronto

#### Use geopy library to get the latitude and longitude values of Toronto City.

Create a map of Toronto using folium

In [15]:
from geopy.geocoders import Nominatim

address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [16]:
import folium
map_toto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Table_final['Latitude'], Table_final['Longitude'], Table_final['Borough'], Table_final['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toto)  
    
map_toto

#### Extracting Boroughs which contain the name Toronto

In [17]:
df_toto=Table_final[Table_final["Borough"].str.contains("Toronto")].reset_index(drop=True)
df_toto.shape

(38, 5)

In [18]:
df_toto

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049


In [19]:
map1_toto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toto['Latitude'], df_toto['Longitude'], df_toto['Borough'], df_toto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map1_toto)  
    
map1_toto

### Explore Neighbourhoods and segment them

In [20]:
CLIENT_ID = 'K13UBXCOAXC5CGGTQ1ZEDYOURT25WU4CK4E4GNFAY0YOQWVY' # your Foursquare ID
CLIENT_SECRET = 'APKBGFFKHQ5K0ZJON5OV4GL3WHQTUHQUD5VUZGEUX3SQFORW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: K13UBXCOAXC5CGGTQ1ZEDYOURT25WU4CK4E4GNFAY0YOQWVY
CLIENT_SECRET:APKBGFFKHQ5K0ZJON5OV4GL3WHQTUHQUD5VUZGEUX3SQFORW


Pick any neighborhood and explore location using foursquare api

In [21]:
neighborhood_latitude = df_toto['Latitude'][0]
neighborhood_longitude = df_toto['Longitude'][0]
neighborhood_name = df_toto['Neighbourhood'][0]

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


Now let's get the top 100 venues in a distance of 1 km = 1000 m

In [22]:
# type your answer here

LIMIT=100
radius=1000

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=K13UBXCOAXC5CGGTQ1ZEDYOURT25WU4CK4E4GNFAY0YOQWVY&client_secret=APKBGFFKHQ5K0ZJON5OV4GL3WHQTUHQUD5VUZGEUX3SQFORW&v=20180605&ll=43.67635739999999,-79.2930312&radius=1000&limit=100'

In [23]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c453ca79fb6b726628efa53'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 78,
  'suggestedBounds': {'ne': {'lat': 43.685357409000005,
    'lng': -79.28061062898105},
   'sw': {'lat': 43.66735739099998, 'lng': -79.30545177101895}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4f5a855be4b0a4baa1ae0063',
       'name': "Tori's Bakeshop",
       'location': {'address': '2188 Queen Street E',
        'crossStreet': 'Balsam Ave',
        'lat': 43.672113947269565,
        'lng': -79.29033140068843,
        'labeledLatLngs

In [24]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean json file and structure into pandas df

In [33]:
#import json
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Tori's Bakeshop,Vegetarian / Vegan Restaurant,43.672114,-79.290331
1,Beaches Bake Shop,Bakery,43.680363,-79.289692
2,Ed's Real Scoop,Ice Cream Shop,43.672630,-79.287993
3,The Fox Theatre,Indie Movie Theater,43.672801,-79.287272
4,The Beech Tree,Gastropub,43.680493,-79.288846


In [34]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

78 venues were returned by Foursquare.


Function to rturn nearby venues to all the neighborhoods in Toronto

In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [39]:
toronto_venues = getNearbyVenues(names=df_toto['Neighbourhood'],
                                   latitudes=df_toto['Latitude'],
                                   longitudes=df_toto['Longitude']
                                  )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvall

In [56]:
print(toronto_venues.shape)
toronto_venues.head()

(3080, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
1,The Beaches,43.676357,-79.293031,Beaches Bake Shop,43.680363,-79.289692,Bakery
2,The Beaches,43.676357,-79.293031,Ed's Real Scoop,43.672630,-79.287993,Ice Cream Shop
3,The Beaches,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater
4,The Beaches,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub


In [57]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,100,100,100,100,100,100
"Brockton,Exhibition Place,Parkdale Village",100,100,100,100,100,100
Business Reply Mail Processing Centre 969 Eastern,47,47,47,47,47,47
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",16,16,16,16,16,16
"Cabbagetown,St. James Town",42,42,42,42,42,42
Central Bay Street,100,100,100,100,100,100
"Chinatown,Grange Park,Kensington Market",100,100,100,100,100,100
Christie,100,100,100,100,100,100


In [58]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 274 unique categories.


#### Analyze individual neighborhoods

In [59]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
toronto_onehot.shape

(3080, 274)

In [61]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Zoo,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.00,0.00,0.00,0.00,0.0000,0.0000,0.040000,0.00,0.00,...,0.01,0.010000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.010000
1,Berczy Park,0.00,0.00,0.00,0.00,0.0000,0.0000,0.010000,0.00,0.00,...,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000
2,"Brockton,Exhibition Place,Parkdale Village",0.00,0.01,0.00,0.00,0.0000,0.0000,0.010000,0.00,0.00,...,0.00,0.030000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.00,0.00,0.00,0.00,0.0000,0.0000,0.021277,0.00,0.00,...,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.00,0.00,0.00,0.00,0.0625,0.0625,0.000000,0.00,0.00,...,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000
5,"Cabbagetown,St. James Town",0.00,0.00,0.00,0.00,0.0000,0.0000,0.023810,0.00,0.00,...,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000
6,Central Bay Street,0.00,0.00,0.00,0.00,0.0000,0.0000,0.010000,0.00,0.00,...,0.01,0.020000,0.00,0.00,0.000000,0.00,0.010000,0.00,0.000000,0.010000
7,"Chinatown,Grange Park,Kensington Market",0.00,0.00,0.01,0.00,0.0000,0.0000,0.000000,0.00,0.00,...,0.00,0.070000,0.00,0.00,0.030000,0.00,0.010000,0.00,0.000000,0.010000
8,Christie,0.00,0.00,0.00,0.00,0.0000,0.0000,0.010000,0.00,0.00,...,0.00,0.020000,0.00,0.00,0.010000,0.00,0.010000,0.00,0.000000,0.000000
9,Church and Wellesley,0.00,0.00,0.01,0.01,0.0000,0.0000,0.020000,0.00,0.00,...,0.00,0.010000,0.01,0.01,0.010000,0.00,0.000000,0.01,0.000000,0.010000


In [62]:
toronto_grouped.shape

(38, 274)

#### Let's print each neighborhood along with the top 5 most common venues

In [63]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0                 Café  0.07
1  American Restaurant  0.04
2              Theater  0.04
3                Hotel  0.04
4          Coffee Shop  0.04


----Berczy Park----
                 venue  freq
0                Hotel  0.07
1          Coffee Shop  0.06
2                 Café  0.05
3           Restaurant  0.05
4  Japanese Restaurant  0.04


----Brockton,Exhibition Place,Parkdale Village----
                    venue  freq
0                    Café  0.06
1             Coffee Shop  0.05
2  Furniture / Home Store  0.04
3              Restaurant  0.04
4                     Bar  0.04


----Business Reply Mail Processing Centre 969 Eastern----
              venue  freq
0              Park  0.11
1       Pizza Place  0.06
2       Coffee Shop  0.06
3     Burrito Place  0.04
4  Sushi Restaurant  0.04


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [64]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [102]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Café,Theater,Coffee Shop,American Restaurant,Hotel,Restaurant,Concert Hall,Japanese Restaurant,Sushi Restaurant,Cosmetics Shop
1,Berczy Park,Hotel,Coffee Shop,Restaurant,Café,Japanese Restaurant,Italian Restaurant,Pub,Cocktail Bar,Bakery,Park
2,"Brockton,Exhibition Place,Parkdale Village",Café,Coffee Shop,Bar,Furniture / Home Store,Restaurant,Vegetarian / Vegan Restaurant,Bakery,Hotel,Arts & Crafts Store,BBQ Joint
3,Business Reply Mail Processing Centre 969 Eastern,Park,Coffee Shop,Pizza Place,Brewery,Burrito Place,Sushi Restaurant,Italian Restaurant,Liquor Store,Pub,Steakhouse
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Coffee Shop,Harbor / Marina,Café,Scenic Lookout,Sculpture Garden,Dog Run,Park,Track,Dance Studio,Tunnel


Run *k*-means to cluster the neighborhoods.

In [103]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 2, 1, 0, 4, 2, 2, 2, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [104]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Coffee Shop,Pub,Pizza Place,Beach,Bar,Breakfast Spot,Japanese Restaurant,Park,Caribbean Restaurant,Bakery
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Pub,Fast Food Restaurant,Café,Pizza Place,Ice Cream Shop,Spa,Ramen Restaurant,Bookstore
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,2,Indian Restaurant,Café,Coffee Shop,Beach,Park,Pizza Place,Brewery,Bakery,Burrito Place,Fast Food Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Coffee Shop,Bar,Bakery,American Restaurant,Café,Diner,Italian Restaurant,Vietnamese Restaurant,Brewery,Park
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Café,College Quad,Trail,College Gym,Coffee Shop,Park,Bookstore,Gym / Fitness Center,Gym,Harbor / Marina


Finally, let's visualize the resulting clusters

In [111]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examine clusters

Cluster0

In [106]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Downtown Toronto,0,Coffee Shop,Harbor / Marina,Café,Scenic Lookout,Sculpture Garden,Dog Run,Park,Track,Dance Studio,Tunnel


Cluster0: Harbor area

#### Examine clusters

Cluster1

In [107]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,1,Coffee Shop,Pub,Pizza Place,Beach,Bar,Breakfast Spot,Japanese Restaurant,Park,Caribbean Restaurant,Bakery
1,East Toronto,1,Greek Restaurant,Coffee Shop,Pub,Fast Food Restaurant,Café,Pizza Place,Ice Cream Shop,Spa,Ramen Restaurant,Bookstore
5,Central Toronto,1,Coffee Shop,Fast Food Restaurant,Italian Restaurant,Café,Pizza Place,Sushi Restaurant,Dessert Shop,Diner,Salad Place,Sporting Goods Shop
6,Central Toronto,1,Sporting Goods Shop,Coffee Shop,Italian Restaurant,Diner,Café,Mexican Restaurant,Fast Food Restaurant,Salon / Barbershop,Spa,Flower Shop
7,Central Toronto,1,Italian Restaurant,Coffee Shop,Sushi Restaurant,Indian Restaurant,Café,Pizza Place,Gym,Thai Restaurant,Restaurant,Mexican Restaurant
8,Central Toronto,1,Coffee Shop,Italian Restaurant,Park,Gym,Café,Playground,Thai Restaurant,Pizza Place,Pub,Sandwich Place
9,Central Toronto,1,Coffee Shop,Park,Sushi Restaurant,Italian Restaurant,Thai Restaurant,Café,Pizza Place,Gym,Gym / Fitness Center,Sandwich Place
10,Downtown Toronto,1,Grocery Store,Coffee Shop,Bank,Park,Convenience Store,Athletics & Sports,Smoothie Shop,Filipino Restaurant,Metro Station,Breakfast Spot
12,Downtown Toronto,1,Coffee Shop,Japanese Restaurant,Burger Joint,Gay Bar,Sushi Restaurant,Park,Café,Gastropub,Diner,Dance Studio
13,Downtown Toronto,1,Coffee Shop,Café,Italian Restaurant,Diner,Breakfast Spot,Sushi Restaurant,Bakery,Theater,Pub,Restaurant


Cluster1: Possible residential areas as there are Banks, Restaurants, Cafes, Diners, Parks, Pharmacy etc.

#### Examine clusters

Cluster2

In [108]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,East Toronto,2,Indian Restaurant,Café,Coffee Shop,Beach,Park,Pizza Place,Brewery,Bakery,Burrito Place,Fast Food Restaurant
3,East Toronto,2,Coffee Shop,Bar,Bakery,American Restaurant,Café,Diner,Italian Restaurant,Vietnamese Restaurant,Brewery,Park
17,Downtown Toronto,2,Coffee Shop,Café,Burger Joint,Ramen Restaurant,Italian Restaurant,Burrito Place,Diner,Tea Room,Concert Hall,Thai Restaurant
24,Central Toronto,2,Café,Vegetarian / Vegan Restaurant,Italian Restaurant,Restaurant,Museum,Bakery,Pizza Place,Gym,Grocery Store,Coffee Shop
25,Downtown Toronto,2,Café,Bakery,Bar,Vegetarian / Vegan Restaurant,Restaurant,Bookstore,Mexican Restaurant,Burrito Place,Caribbean Restaurant,Thai Restaurant
26,Downtown Toronto,2,Café,Bar,Vegetarian / Vegan Restaurant,Bakery,Coffee Shop,Vietnamese Restaurant,Mexican Restaurant,Tea Room,Dessert Shop,Dumpling Restaurant
30,Downtown Toronto,2,Korean Restaurant,Coffee Shop,Café,Grocery Store,Ice Cream Shop,Indian Restaurant,Cocktail Bar,Pizza Place,Bar,Japanese Restaurant
31,West Toronto,2,Café,Park,Coffee Shop,Supermarket,Bar,Discount Store,Brewery,Sushi Restaurant,Portuguese Restaurant,Bakery
32,West Toronto,2,Café,Bar,Restaurant,Bakery,Coffee Shop,Cocktail Bar,Pizza Place,Italian Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant
33,West Toronto,2,Café,Coffee Shop,Bar,Furniture / Home Store,Restaurant,Vegetarian / Vegan Restaurant,Bakery,Hotel,Arts & Crafts Store,BBQ Joint


Cluster2: Cafe and coffee shops

#### Examine clusters

Cluster3

In [109]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,3,Café,College Quad,Trail,College Gym,Coffee Shop,Park,Bookstore,Gym / Fitness Center,Gym,Harbor / Marina


Cluster3: Education and fitness Hub in Central Toronto

#### Examine clusters

Cluster4

In [110]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Downtown Toronto,4,Café,Restaurant,Park,Coffee Shop,Gastropub,Diner,Italian Restaurant,Japanese Restaurant,Pub,Taiwanese Restaurant
14,Downtown Toronto,4,Coffee Shop,Clothing Store,Restaurant,Café,Italian Restaurant,Ramen Restaurant,Tea Room,Plaza,Gastropub,Middle Eastern Restaurant
15,Downtown Toronto,4,Coffee Shop,Restaurant,Café,Hotel,Italian Restaurant,Bakery,Breakfast Spot,Seafood Restaurant,Cosmetics Shop,Gastropub
16,Downtown Toronto,4,Hotel,Coffee Shop,Restaurant,Café,Japanese Restaurant,Italian Restaurant,Pub,Cocktail Bar,Bakery,Park
18,Downtown Toronto,4,Café,Theater,Coffee Shop,American Restaurant,Hotel,Restaurant,Concert Hall,Japanese Restaurant,Sushi Restaurant,Cosmetics Shop
19,Downtown Toronto,4,Coffee Shop,Hotel,Café,Aquarium,Brewery,Scenic Lookout,Park,Italian Restaurant,Deli / Bodega,Restaurant
20,Downtown Toronto,4,Hotel,Café,Coffee Shop,Italian Restaurant,Restaurant,Steakhouse,Concert Hall,American Restaurant,Gym,Sports Bar
21,Downtown Toronto,4,Hotel,Coffee Shop,Café,Japanese Restaurant,Gastropub,Beer Bar,Concert Hall,American Restaurant,Steakhouse,Restaurant
28,Downtown Toronto,4,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Gastropub,Cocktail Bar,BBQ Joint
29,Downtown Toronto,4,Hotel,Café,Coffee Shop,American Restaurant,Restaurant,Italian Restaurant,Theater,Japanese Restaurant,Steakhouse,Gastropub


Cluster4: Downtown Food and Entertainment